<a href="https://colab.research.google.com/github/diogomattos/Kaggle-Bank-Marketing-ETL/blob/main/desafio_etl_kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Instalar bibliotecas necessárias
!pip install google-cloud-bigquery pandas kaggle

# Importar bibliotecas
import pandas as pd
from google.cloud import bigquery
from google.colab import auth

# Autenticar com Google Cloud
auth.authenticate_user()

# Configurar credenciais do Google Cloud
from google.cloud import bigquery
client = bigquery.Client()

# Configurar Kaggle
import os
os.environ['KAGGLE_USERNAME'] = 'dogmaistoot'
os.environ['KAGGLE_KEY'] = '10ecd5878d1d2851da330a0ff6a37504'


In [21]:
# Baixar dados do Kaggle
!kaggle datasets download -d prakharrathi25/banking-dataset-marketing-targets
!unzip banking-dataset-marketing-targets.zip -d ./banking_dataset

# Carregar dados de treino e teste para o Colab com delimitador correto
df_train = pd.read_csv('./banking_dataset/train.csv', delimiter=';')
df_test = pd.read_csv('./banking_dataset/test.csv', delimiter=';')

# Verificar as primeiras linhas e as colunas do DataFrame
print(df_train.head())
print(df_train.columns)

# Limpeza dos Dados
def clean_data(df):
    # Avaliar qualidade dos dados
    print(df.isnull().sum())
    print(df.duplicated().sum())

    # Limpar dados (exemplo)
    df.dropna(inplace=True)
    df.drop_duplicates(inplace=True)

    # Transformar dados conforme necessário
    df['age'] = df['age'].astype(int)
    df['balance'] = df['balance'].astype(float)

    # Converter colunas categóricas para tipos de dados apropriados
    categorical_columns = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome', 'y']
    for column in categorical_columns:
        df[column] = df[column].astype('category')

    return df

df_train = clean_data(df_train)
df_test = clean_data(df_test)

print(df_train.head())
print(df_test.head())


Dataset URL: https://www.kaggle.com/datasets/prakharrathi25/banking-dataset-marketing-targets
License(s): CC0-1.0
banking-dataset-marketing-targets.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  banking-dataset-marketing-targets.zip
replace ./banking_dataset/test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ./banking_dataset/test.csv  
replace ./banking_dataset/train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ./banking_dataset/train.csv  
   age           job  marital  education default  balance housing loan  \
0   58    management  married   tertiary      no     2143     yes   no   
1   44    technician   single  secondary      no       29     yes   no   
2   33  entrepreneur  married  secondary      no        2     yes  yes   
3   47   blue-collar  married    unknown      no     1506     yes   no   
4   33       unknown   single    unknown      no        1      no   no   

   contact  day month  duration  c

In [22]:
# Importar bibliotecas adicionais
import pandas as pd
import numpy as np
from google.cloud import bigquery
from google.cloud.exceptions import NotFound

# Função para carregar dados em partes menores
def load_data_in_chunks(dataframe, table_id, client, schema, chunk_size=10000):
    chunks = np.array_split(dataframe, len(dataframe) // chunk_size + 1)
    job_config = bigquery.LoadJobConfig(
        schema=schema,
        write_disposition=bigquery.WriteDisposition.WRITE_APPEND
    )

    for i, chunk in enumerate(chunks):
        print(f"Carregando chunk {i + 1} de {len(chunks)}...")
        try:
            job = client.load_table_from_dataframe(chunk, table_id, job_config=job_config)
            job.result()  # Espera o job terminar
            print(f"Chunk {i + 1} carregado com sucesso.")
        except Exception as e:
            print(f"Erro ao carregar o chunk {i + 1}: {e}")

# Definir esquema da tabela
schema = [
    bigquery.SchemaField("age", "INTEGER"),
    bigquery.SchemaField("job", "STRING"),
    bigquery.SchemaField("marital", "STRING"),
    bigquery.SchemaField("education", "STRING"),
    bigquery.SchemaField("default", "STRING"),
    bigquery.SchemaField("balance", "FLOAT"),
    bigquery.SchemaField("housing", "STRING"),
    bigquery.SchemaField("loan", "STRING"),
    bigquery.SchemaField("contact", "STRING"),
    bigquery.SchemaField("day", "INTEGER"),
    bigquery.SchemaField("month", "STRING"),
    bigquery.SchemaField("duration", "INTEGER"),
    bigquery.SchemaField("campaign", "INTEGER"),
    bigquery.SchemaField("pdays", "INTEGER"),
    bigquery.SchemaField("previous", "INTEGER"),
    bigquery.SchemaField("poutcome", "STRING"),
    bigquery.SchemaField("y", "STRING"),
]

# Substitua 'bq-study-288301.estudos_kaggle' pelo ID do seu projeto e dataset
project_id = "bq-study-288301"
dataset_id = "estudos_kaggle"
table_train_id = f"{project_id}.{dataset_id}.bank_marketing_train"
table_test_id = f"{project_id}.{dataset_id}.bank_marketing_test"

# Criar cliente do BigQuery com o project_id
client = bigquery.Client(project=project_id)

# Verificar se o dataset existe
dataset_ref = client.dataset(dataset_id)
try:
    client.get_dataset(dataset_ref)  # Make an API request.
    print(f"Dataset {dataset_id} encontrado.")
except NotFound:
    print(f"Dataset {dataset_id} não encontrado.")
    dataset = bigquery.Dataset(dataset_ref)
    dataset.location = "US"
    client.create_dataset(dataset)  # Make an API request.
    print(f"Dataset {dataset_id} criado.")

# Carregar dados no BigQuery em partes menores
load_data_in_chunks(df_train, table_train_id, client, schema)
load_data_in_chunks(df_test, table_test_id, client, schema)

print(f"Carregado {len(df_train)} linhas na tabela {table_train_id}.")
print(f"Carregado {len(df_test)} linhas na tabela {table_test_id}.")


Dataset estudos_kaggle encontrado.
Carregando chunk 1 de 5...
Chunk 1 carregado com sucesso.
Carregando chunk 2 de 5...
Chunk 2 carregado com sucesso.
Carregando chunk 3 de 5...
Chunk 3 carregado com sucesso.
Carregando chunk 4 de 5...
Chunk 4 carregado com sucesso.
Carregando chunk 5 de 5...
Chunk 5 carregado com sucesso.
Carregando chunk 1 de 1...
Chunk 1 carregado com sucesso.
Carregado 45211 linhas na tabela bq-study-288301.estudos_kaggle.bank_marketing_train.
Carregado 4521 linhas na tabela bq-study-288301.estudos_kaggle.bank_marketing_test.


In [19]:
# Importar bibliotecas adicionais
import pandas as pd
import numpy as np
from google.cloud import bigquery
from google.cloud.exceptions import NotFound

# Função para carregar dados em partes menores
def load_data_in_chunks(dataframe, table_id, client, schema, chunk_size=10000):
    chunks = np.array_split(dataframe, len(dataframe) // chunk_size + 1)
    job_config = bigquery.LoadJobConfig(
        schema=schema,
        write_disposition=bigquery.WriteDisposition.WRITE_APPEND
    )

    for i, chunk in enumerate(chunks):
        print(f"Carregando chunk {i + 1} de {len(chunks)}...")
        try:
            job = client.load_table_from_dataframe(chunk, table_id, job_config=job_config)
            job.result()  # Espera o job terminar
            print(f"Chunk {i + 1} carregado com sucesso.")
        except Exception as e:
            print(f"Erro ao carregar o chunk {i + 1}: {e}")

# Definir esquema da tabela
schema = [
    bigquery.SchemaField("age", "INTEGER"),
    bigquery.SchemaField("job", "STRING"),
    bigquery.SchemaField("marital", "STRING"),
    bigquery.SchemaField("education", "STRING"),
    bigquery.SchemaField("default", "STRING"),
    bigquery.SchemaField("balance", "FLOAT"),
    bigquery.SchemaField("housing", "STRING"),
    bigquery.SchemaField("loan", "STRING"),
    bigquery.SchemaField("contact", "STRING"),
    bigquery.SchemaField("day", "INTEGER"),
    bigquery.SchemaField("month", "STRING"),
    bigquery.SchemaField("duration", "INTEGER"),
    bigquery.SchemaField("campaign", "INTEGER"),
    bigquery.SchemaField("pdays", "INTEGER"),
    bigquery.SchemaField("previous", "INTEGER"),
    bigquery.SchemaField("poutcome", "STRING"),
    bigquery.SchemaField("y", "STRING"),
]

# Substitua 'bq-study-288301.estudos_kaggle' pelo ID do seu projeto e dataset
project_id = "bq-study-288301"
dataset_id = "estudos_kaggle"
table_id = f"{project_id}.{dataset_id}.bank_marketing"

# Criar cliente do BigQuery com o project_id
client = bigquery.Client(project=project_id)

# Verificar se o dataset existe
dataset_ref = client.dataset(dataset_id)
try:
    client.get_dataset(dataset_ref)  # Make an API request.
    print(f"Dataset {dataset_id} encontrado.")
except NotFound:
    print(f"Dataset {dataset_id} não encontrado.")
    dataset = bigquery.Dataset(dataset_ref)
    dataset.location = "US"
    client.create_dataset(dataset)  # Make an API request.
    print(f"Dataset {dataset_id} criado.")

# Carregar dados no BigQuery em partes menores
load_data_in_chunks(df_train, table_id, client, schema)

print(f"Carregado {len(df_train)} linhas na tabela {table_id}.")


Dataset estudos_kaggle encontrado.
Carregando chunk 1 de 5...
Chunk 1 carregado com sucesso.
Carregando chunk 2 de 5...
Chunk 2 carregado com sucesso.
Carregando chunk 3 de 5...
Chunk 3 carregado com sucesso.
Carregando chunk 4 de 5...
Chunk 4 carregado com sucesso.
Carregando chunk 5 de 5...
Chunk 5 carregado com sucesso.
Carregado 45211 linhas na tabela bq-study-288301.estudos_kaggle.bank_marketing.


In [23]:
# Exemplo de consulta SQL para dados de treino
query_train = f"""
SELECT job, COUNT(*) as job_count
FROM `{table_train_id}`
GROUP BY job
ORDER BY job_count DESC
"""
query_job_train = client.query(query_train)

print("Dados de Treinamento:")
for row in query_job_train:
    print(f"{row.job}: {row.job_count}")

# Exemplo de consulta SQL para dados de teste
query_test = f"""
SELECT job, COUNT(*) as job_count
FROM `{table_test_id}`
GROUP BY job
ORDER BY job_count DESC
"""
query_job_test = client.query(query_test)

print("Dados de Teste:")
for row in query_job_test:
    print(f"{row.job}: {row.job_count}")


Dados de Treinamento:
blue-collar: 9732
management: 9458
technician: 7597
admin.: 5171
services: 4154
retired: 2264
self-employed: 1579
entrepreneur: 1487
unemployed: 1303
housemaid: 1240
student: 938
unknown: 288
Dados de Teste:
management: 969
blue-collar: 946
technician: 768
admin.: 478
services: 417
retired: 230
self-employed: 183
entrepreneur: 168
unemployed: 128
housemaid: 112
student: 84
unknown: 38
